In [1]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from sklearn.base import clone
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import gc

warnings.filterwarnings("ignore")
import os
os.chdir("/Users/deweywang/Desktop/GitHub/FlightRank/")


In [10]:
import pandas as pd
import os

model_dir = "model_output/all_features_xgb"
label = "xlarge"
model_importance_dir = os.path.join(model_dir, "model_importance")
csv_path = os.path.join(model_importance_dir, f"feature_importance_{label}_all_features.csv")

# 讀取
df = pd.read_csv(csv_path)

# 第幾名開始沒用過
idx_first_unused = df[df["min_rank"] == 9999].index.min()

print(f"\n✅ 第 {idx_first_unused} 名後都是完全未使用的特徵\n")

# 看第一個
print("✅ 第一個未使用特徵：")
print(df.iloc[idx_first_unused])


features = (df.iloc[:idx_first_unused]['feature'].tolist())



✅ 第 78 名後都是完全未使用的特徵

✅ 第一個未使用特徵：
feature            legs1_segments3_marketingCarrier_code
weight                                               NaN
weight_rank_pos                                   9999.0
gain                                                 NaN
gain_rank_pos                                     9999.0
cover                                                NaN
cover_rank_pos                                    9999.0
min_rank                                          9999.0
Name: 78, dtype: object


In [25]:
import polars as pl

def split_data_by_group_size_per_label_features(
    df: pl.DataFrame,
    bins: list,
    labels: list,
    label_features: dict = None
):
    """
    一次切分所有分群，每個分群用各自features。
    
    df: polars.DataFrame
    bins: 分群邊界
    labels: 分群名稱
    label_features: dict(label -> feature list)，每個label自己的feature欄位。
                   如果為None，則所有分群都用df的所有columns。
    """
    df = df.with_row_count("global_row_nr")

    group_counts = (
        df.group_by("ranker_id")
          .agg(pl.count().alias("n_rows"))
          .filter(pl.col("n_rows") >= bins[0])
    )

    bins_fixed = bins.copy()
    if bins_fixed[-1] is None:
        max_value = group_counts["n_rows"].max()
        bins_fixed[-1] = int(max_value) + 1

    if len(labels) != len(bins_fixed) - 1:
        raise ValueError(f"bins={bins_fixed} 有 {len(bins_fixed)-1}個區間，但labels數={len(labels)}")

    cond = (
        pl.when((pl.col("n_rows") >= bins_fixed[0]) & (pl.col("n_rows") < bins_fixed[1]))
        .then(pl.lit(labels[0]))
    )
    for i in range(1, len(labels)):
        cond = cond.when(
            (pl.col("n_rows") >= bins_fixed[i]) & (pl.col("n_rows") < bins_fixed[i+1])
        ).then(pl.lit(labels[i]))
    cond = cond.otherwise(pl.lit("unknown"))

    group_counts = group_counts.with_columns([
        cond.alias("group_category")
    ])

    df = df.join(group_counts, on="ranker_id", how="left")

    split_data = {}
    for lbl in labels:
        subset = df.filter(pl.col("group_category") == lbl)

        # 如果沒給 label_features，就用全部 columns
        if label_features is None:
            feats = [c for c in df.columns if c not in ("group_category")]
        else:
            feats = label_features.get(lbl, [])

        base_cols = ["selected", "ranker_id", "global_row_nr", "group_category"]
        all_cols = feats + base_cols
        all_cols = list(dict.fromkeys(all_cols))  # 去重防止重複

        subset = subset.select([c for c in all_cols if c in subset.columns])

        mem_mb = subset.estimated_size() / (1024 * 1024)
        print(f"✅ {lbl}: {subset.height} rows, approx {mem_mb:.2f} MB")
        split_data[lbl] = subset

    summary = (
        group_counts.group_by("group_category")
        .agg([
            pl.count().alias("n_groups"),
            pl.col("n_rows").sum().alias("total_rows"),
            pl.col("n_rows").mean().alias("avg_rows_per_group")
        ])
        .sort("group_category")
    )

    print("✅ 分群統計：")
    print(summary)

    return {
        "data_all": df,
        "split_data": split_data,
        "summary": summary
    }


In [26]:
import os
import pandas as pd

def load_label_features(
    model_dir: str,
    split_labels: list,
    top_n: int = None
):
    """
    從模型資料夾讀取每個分群的特徵重要性檔案，回傳每個label的features list。
    
    參數:
    - model_dir: 模型資料夾
    - split_labels: 分群名稱list
    - top_n: 如果指定，只取前N個特徵；否則用第一個min_rank=9999為止

    回傳:
    - dict(label -> features list)
    """
    label_features = {}

    for label in split_labels:
        model_importance_dir = os.path.join(model_dir, "model_importance")
        csv_path = os.path.join(model_importance_dir, f"feature_importance_{label}_all_features.csv")

        df = pd.read_csv(csv_path)

        if top_n is not None:
            # 直接取前top_n
            feats = df.iloc[:top_n]["feature"].tolist()
            print(f"\n✅ {label}: 取前 {top_n} 個特徵")
        else:
            idx_first_unused = df[df["min_rank"] == 9999].index.min()
            feats = df.iloc[:idx_first_unused]["feature"].tolist()
            print(f"\n✅ {label}: 第 {idx_first_unused} 名後都是完全未使用的特徵")
            print("✅ 第一個未使用特徵：")
            print(df.iloc[idx_first_unused])

        label_features[label] = feats

    # 印出所有分群features數量
    for label in split_labels:
        print(f"{label}: {len(label_features[label])} features")

    return label_features


In [32]:
import numpy as np
import polars as pl
import pandas as pd
import os

model_dir = "model_output/all_features_xgb"

# 先把每個分群都讀一遍feature importance
split_labels = ["small", "medium", "large"]

label_features = load_label_features(
    model_dir=model_dir,
    split_labels=split_labels,
    top_n=20
)

# 讀 train data
train_filled = pl.read_parquet("data/train_filled.parquet")

# 呼叫新版本分群切分
result = split_data_by_group_size_per_label_features(
    df=train_filled,
    bins=[3, 27, 162,None],
    labels=split_labels,
    label_features=label_features
)

# 清理記憶體
import gc
del train_filled
gc.collect()



✅ small: 取前 20 個特徵

✅ medium: 取前 20 個特徵

✅ large: 取前 20 個特徵
small: 20 features
medium: 20 features
large: 20 features
✅ small: 416866 rows, approx 66.84 MB
✅ medium: 3127131 rows, approx 480.52 MB
✅ large: 14595402 rows, approx 2340.18 MB
✅ 分群統計：
shape: (3, 4)
┌────────────────┬──────────┬────────────┬────────────────────┐
│ group_category ┆ n_groups ┆ total_rows ┆ avg_rows_per_group │
│ ---            ┆ ---      ┆ ---        ┆ ---                │
│ str            ┆ u32      ┆ u32        ┆ f64                │
╞════════════════╪══════════╪════════════╪════════════════════╡
│ large          ┆ 25590    ┆ 14595402   ┆ 570.355686         │
│ medium         ┆ 45114    ┆ 3127131    ┆ 69.316199          │
│ small          ┆ 31288    ┆ 416866     ┆ 13.323511          │
└────────────────┴──────────┴────────────┴────────────────────┘


0

In [33]:
import os
import gc
import xgboost as xgb
import numpy as np
import polars as pl
from scripts.group_wise import prepare_train_val_split
from scripts.hitrate import compute_hitrate_at_3

# 自訂模型儲存目錄
model_dir = "model_output/selected_features_xgb/v2_top20/"
os.makedirs(model_dir, exist_ok=True)
print(f"✅ Model folder: {model_dir}")

# 預設 XGB 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

split_labels = ["small", "medium", "large"]

split_data = result["split_data"]

# 建立 train_dataset 子資料夾
train_dataset_dir = os.path.join(model_dir, "train_dataset")
os.makedirs(train_dataset_dir, exist_ok=True)

# 存 parquet 到 train_dataset 資料夾
for label, df in split_data.items():
    path = os.path.join(train_dataset_dir, f"{label}.parquet")
    print(f"💾 儲存 {label} 到 {path}")
    df.write_parquet(path)


# 清理記憶體
split_data.clear()
del result
gc.collect()

print("✅ split_data 已清空，記憶體應已釋放")

all_groups_val_np = []
all_y_val_np = []
all_val_preds = []
hitrate_per_group = {}

for label in split_labels:
    print(f"\n🚀 處理分群: {label}")

    data = pl.read_parquet(os.path.join(train_dataset_dir, f"{label}.parquet"))

    split_result = prepare_train_val_split(
        result={"split_data": {label: data}},
        split_label=label,
        feature_cols=label_features[label],
        train_fraction=0.8
    )

    dtrain = xgb.DMatrix(
        split_result["X_train_np"],
        label=split_result["y_train_np"]
    )
    dtrain.set_group(split_result["group_sizes_train"])

    dval = xgb.DMatrix(
        split_result["X_val_np"],
        label=split_result["y_val_np"]
    )
    dval.set_group(split_result["group_sizes_val"])

    xgb_model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=500,
        evals=[(dtrain, "train"), (dval, "val")],
        early_stopping_rounds=50,
        verbose_eval=20
    )

    model_path = os.path.join(model_dir, f"xgb_ranker_{label}.bin")
    xgb_model.save_model(model_path)
    print(f"✅ 已儲存模型: {model_path}")

    val_preds = xgb_model.predict(dval)

    hitrate = compute_hitrate_at_3(
        split_result["groups_val_np"],
        split_result["y_val_np"],
        val_preds
    )
    hitrate_per_group[label] = hitrate

    all_groups_val_np.append(split_result["groups_val_np"])
    all_y_val_np.append(split_result["y_val_np"])
    all_val_preds.append(val_preds)

    del dtrain, dval, xgb_model, val_preds, split_result, data
    gc.collect()
    print("🧹 記憶體已清理")

print("\n⚡ 合併所有驗證結果計算 HitRate ⚡")
all_groups_val_np = np.concatenate(all_groups_val_np)
all_y_val_np = np.concatenate(all_y_val_np)
all_val_preds = np.concatenate(all_val_preds)

overall_hitrate = compute_hitrate_at_3(
    all_groups_val_np,
    all_y_val_np,
    all_val_preds
)

print("\n💡 每組 Hitrate：")
for label, h in hitrate_per_group.items():
    print(f"{label}: {h:.4f}")

print(f"\n🎯 全部合併 HitRate: {overall_hitrate:.4f}")

# 將 hitrate 寫入 CSV
hitrate_records = [
    {"split_label": label, "hitrate": h}
    for label, h in hitrate_per_group.items()
]
hitrate_records.append({"split_label": "overall", "hitrate": overall_hitrate})

hitrate_df = pl.DataFrame(hitrate_records)
csv_path = os.path.join(model_dir, "hitrate_summary.csv")
hitrate_df.write_csv(csv_path)
print(f"\n✅ 已儲存所有 Hitrate 結果至 {csv_path}")


✅ Model folder: model_output/selected_features_xgb/v2_top20/
💾 儲存 small 到 model_output/selected_features_xgb/v2_top20/train_dataset/small.parquet
💾 儲存 medium 到 model_output/selected_features_xgb/v2_top20/train_dataset/medium.parquet
💾 儲存 large 到 model_output/selected_features_xgb/v2_top20/train_dataset/large.parquet
✅ split_data 已清空，記憶體應已釋放

🚀 處理分群: small
✅ Train: 333058 rows, 25030 groups
✅ Val: 83808 rows, 6258 groups
[0]	train-ndcg@3:0.67297	val-ndcg@3:0.66570
[20]	train-ndcg@3:0.77250	val-ndcg@3:0.73017
[40]	train-ndcg@3:0.78823	val-ndcg@3:0.73522
[60]	train-ndcg@3:0.79962	val-ndcg@3:0.73904
[80]	train-ndcg@3:0.80932	val-ndcg@3:0.74334
[100]	train-ndcg@3:0.81926	val-ndcg@3:0.74608
[120]	train-ndcg@3:0.82810	val-ndcg@3:0.74955
[140]	train-ndcg@3:0.83695	val-ndcg@3:0.75290
[160]	train-ndcg@3:0.84280	val-ndcg@3:0.75381
[180]	train-ndcg@3:0.85067	val-ndcg@3:0.75518
[200]	train-ndcg@3:0.85743	val-ndcg@3:0.75695
[220]	train-ndcg@3:0.86320	val-ndcg@3:0.75742
[240]	train-ndcg@3:0.86858	val

In [22]:
import os
import xgboost as xgb
import pandas as pd

def export_xgb_feature_importance(
    model_dir: str,
    split_labels: list,
    label_features: dict
    ):
    """
    讀取多個分群的xgb模型，輸出完整feature importance CSV。

    參數:
    - model_dir: 模型資料夾
    - split_labels: 分群label列表 (ex: ["small", "medium", ...])
    - label_features: dict(label -> 該分群使用的特徵list)
    - top_n: 顯示/列印前N重要特徵

    每個分群會輸出一個CSV在 model_importance 子資料夾。
    """
    model_importance_dir = os.path.join(model_dir, "model_importance")
    os.makedirs(model_importance_dir, exist_ok=True)

    for label in split_labels:
        features_this_label = label_features[label]
        print(f"\n🚀 處理分群 {label}... (特徵數: {len(features_this_label)})")

        model_path = os.path.join(model_dir, f"xgb_ranker_{label}.bin")
        booster = xgb.Booster()
        booster.load_model(model_path)
        print(f"✅ 已讀取模型: {model_path}")

        importance_types = ["weight", "gain", "cover"]
        importance_all = {}

        # 取得三種重要性
        for imp_type in importance_types:
            imp_raw = booster.get_score(importance_type=imp_type)
            imp_named = {}
            for k, v in imp_raw.items():
                idx = int(k[1:])
                real_name = features_this_label[idx]
                imp_named[real_name] = v
            sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
            importance_all[imp_type] = sorted_imp

        # 各別DataFrame
        df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight"])
        df_weight["weight_rank_pos"] = df_weight.index

        df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain"])
        df_gain["gain_rank_pos"] = df_gain.index

        df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover"])
        df_cover["cover_rank_pos"] = df_cover.index

        # 建完整feature表
        df_all = pd.DataFrame({"feature": features_this_label})

        # 合併
        df_merged = (
            df_all
            .merge(df_weight, on="feature", how="left")
            .merge(df_gain, on="feature", how="left")
            .merge(df_cover, on="feature", how="left")
        )

        # 補rank_pos
        df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
        df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
        df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

        # 計算最小rank
        df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

        # 排序
        df_merged_sorted = df_merged.sort_values("min_rank")

        # 輸出
        csv_path = os.path.join(model_importance_dir, f"feature_importance_{label}_all_features.csv")
        df_merged_sorted.to_csv(csv_path, index=False)
        print(f"✅ 已輸出特徵重要性到 {csv_path}")


In [24]:
export_xgb_feature_importance(
    model_dir=model_dir,
    split_labels=split_labels,
    label_features=label_features,  # 請用訓練用的特徵
)



🚀 處理分群 small... (特徵數: 95)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_small.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_small_all_features.csv

🚀 處理分群 medium... (特徵數: 99)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_medium.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_medium_all_features.csv

🚀 處理分群 large... (特徵數: 95)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_large.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_large_all_features.csv

🚀 處理分群 xlarge... (特徵數: 78)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_xlarge.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_xlarge_all_features.csv


In [ ]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("train_filled.parquet")

# 讀取top50 features


df_top50 = pd.read_csv("feature_importance_top50.csv")
top50_features = df_top50["feature"].tolist()

# 你要保留的欄位 (Top50 + target + group id)
cols_to_keep = top50_features + ["selected", "ranker_id"]

# 只保留這些欄位
train_filled = train_filled.select(cols_to_keep)

print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 52 欄位: ['totalPrice', 'legs0_segments0_cabinClass', 'companyID', 'legs0_segments1_departureFrom_airport_iata', 'legs0_segments0_marketingCarrier_code', 'legs1_segments1_departureFrom_airport_iata', 'legs0_segments0_flightNumber', 'both_legs_direct', 'legs1_segments1_arrivalTo_airport_city_iata', 'legs1_segments0_flightNumber', 'legs0_main_carrier', 'days_before_departure', 'legs0_segments0_baggageAllowance_quantity', 'legs1_segments0_cabinClass', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs1_segments0_seatsAvailable', 'legs0_segments1_baggageAllowance_quantity', 'legs0_segments0_seatsAvailable', 'miniRules0_monetaryAmount', 'legs0_arrivalAt_hour', 'legs1_segments0_baggageAllowance_quantity', 'legs1_arrivalAt_hour', 'legs0_departureAt_hour', 'legs0_segments1_flightNumber', 'nationality', 'legs1_departureAt_hour', 'legs1_segments1_flightNumber', 'corporateTariffCode', 'legs1_main_carrier', 'legs1_segments0_departureFrom_airport_iata', 'taxes', 'legs1_departureAt_

In [3]:

import polars as pl

# 第一步：計算每個ranker_id的出現次數
group_counts = (
    train_filled
    .group_by("ranker_id")
    .agg(pl.count().alias("n_rows"))
)

# 第二步：挑選 >=10 的 ranker_id
valid_ranker_ids = (
    group_counts
    .filter(pl.col("n_rows") >= 10)
    .select("ranker_id")
)

# 第三步：過濾掉 <10 的
train_filled = (
    train_filled
    .join(valid_ranker_ids, on="ranker_id", how="inner")
)

print(f"刪掉後還有 {train_filled.height} 行資料")


刪掉後還有 18075751 行資料


In [4]:
exclude_cols = [
    'Id', 'ranker_id', 'selected'
]

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

Using 50 features


In [5]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]

# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])
val_rankers = set(unique_rankers[n_train:])

# 用 Polars 過濾 train/val
is_train = groups.select(pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train"))

# 先 concat mask
X_with_mask = X.with_columns(is_train)
y_with_mask = y.with_columns(is_train)
groups_with_mask = groups.with_columns(is_train)

# 分割 DataFrame
X_train_df = X_with_mask.filter(pl.col("is_train"))
X_val_df = X_with_mask.filter(~pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
y_val_df = y_with_mask.filter(~pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))
groups_val_df = groups_with_mask.filter(~pl.col("is_train"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_train").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_train").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_train").to_numpy().flatten()

# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id")
    .agg(pl.len().alias("size"))
    .sort("ranker_id")["size"]
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_train")
    .group_by("ranker_id")
    .agg(pl.len().alias("size"))
    .sort("ranker_id")["size"]
    .to_numpy()
)


In [6]:
import xgboost as xgb

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
)
dtrain.set_group(group_sizes_train)

dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
)
dval.set_group(group_sizes_val)


In [7]:

# 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)


[0]	train-ndcg@3:0.72792	val-ndcg@3:0.73653
[20]	train-ndcg@3:0.77023	val-ndcg@3:0.76656
[40]	train-ndcg@3:0.77818	val-ndcg@3:0.76950
[60]	train-ndcg@3:0.78411	val-ndcg@3:0.77374
[80]	train-ndcg@3:0.78976	val-ndcg@3:0.77621
[100]	train-ndcg@3:0.79574	val-ndcg@3:0.77886
[120]	train-ndcg@3:0.80164	val-ndcg@3:0.78131
[140]	train-ndcg@3:0.80678	val-ndcg@3:0.78428
[160]	train-ndcg@3:0.81189	val-ndcg@3:0.78623
[180]	train-ndcg@3:0.81691	val-ndcg@3:0.78766
[200]	train-ndcg@3:0.82134	val-ndcg@3:0.78926
[220]	train-ndcg@3:0.82588	val-ndcg@3:0.79097
[240]	train-ndcg@3:0.82996	val-ndcg@3:0.79216
[260]	train-ndcg@3:0.83413	val-ndcg@3:0.79359
[280]	train-ndcg@3:0.83805	val-ndcg@3:0.79404
[300]	train-ndcg@3:0.84146	val-ndcg@3:0.79494
[320]	train-ndcg@3:0.84446	val-ndcg@3:0.79546
[340]	train-ndcg@3:0.84737	val-ndcg@3:0.79649
[360]	train-ndcg@3:0.85052	val-ndcg@3:0.79670
[380]	train-ndcg@3:0.85287	val-ndcg@3:0.79780
[400]	train-ndcg@3:0.85523	val-ndcg@3:0.79829
[420]	train-ndcg@3:0.85710	val-ndcg@3:0.

In [22]:
import polars as pl

def compute_hitrate_at_3(groups_np, labels_np, preds_np):
    """
    計算 HitRate@3 (含所有 groups 與 groups >10)
    參數:
        groups_np: np.array of ranker_id
        labels_np: np.array of true labels (0/1)
        preds_np: np.array of predicted scores
    回傳:
        (hitrate_all, hitrate_large_groups)
    """
    # 建立 DataFrame
    val_df = pl.DataFrame({
        "ranker_id": groups_np,
        "label": labels_np,
        "score": preds_np
    })

    # 排序
    val_df = val_df.sort(["ranker_id", "score"], descending=[False, True])

    # rank_in_group
    val_df = val_df.with_columns([
        pl.col("score").rank(method="dense", descending=True).over("ranker_id").alias("rank_in_group")
    ])
    # 對每組找 label=1 的 row
    hit_df = (
        val_df.filter(pl.col("label") == 1)
              .with_columns([
                  (pl.col("rank_in_group") <=3).cast(pl.Int8).alias("is_hit")
              ])
    )

    # 計算每組大小
    group_size_df = (
        val_df.group_by("ranker_id")
              .agg(pl.len().alias("group_size"))
    )

    # Join 回去
    hit_with_size = hit_df.join(group_size_df, on="ranker_id")

    # 過濾 >10
    hit_large_groups = hit_with_size.filter((pl.col("group_size") > 10))

    # HitRate >10 groups
    num_large_groups = hit_large_groups.height
    num_large_hits = hit_large_groups["is_hit"].sum()
    if num_large_groups > 0:
        hitrate_large = num_large_hits / num_large_groups
    else:
        hitrate_large = 0.0

    # 輸出
    print(f"✅ HitRate@3 (groups >10): {hitrate_large:.4f}")

    return hitrate_large
# 預測
val_preds = xgb_model.predict(dval)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)


✅ HitRate@3 (groups >10): 0.5079


0.5079179212668674

In [10]:
y_val_np

array([0, 1, 0, ..., 0, 0, 0], shape=(3769685,))

In [12]:
# 存成二進位模型
xgb_model.save_model("xgb_ranker_model.bin")
print("✅ 已儲存模型到 'xgb_ranker_model.bin'")


✅ 已儲存模型到 'xgb_ranker_model.bin'


In [19]:
# 取得重要性 (index key: f0, f1, ...)
importance_dict = xgb_model.get_score(importance_type="weight")

# 把特徵名稱對應回 X.columns
# 如果X是 Polars DataFrame
feature_names = list(X.columns)

# 重新命名
importance_named = {}
for k, v in importance_dict.items():
    # k = 'f0', 'f1', ...
    idx = int(k[1:])  # 取數字
    real_name = feature_names[idx]
    importance_named[real_name] = v

# 排序
sorted_importance = sorted(importance_named.items(), key=lambda x: x[1], reverse=True)

# 列印
print("⭐ Top 20 Feature Importances (by weight):")
for feat, score in sorted_importance[:20]:
    print(f"{feat}: {score}")


⭐ Top 20 Feature Importances (by weight):
totalPrice: 8578.0
companyID: 6165.0
legs0_segments0_flightNumber: 6092.0
legs1_segments0_flightNumber: 5554.0
days_before_departure: 5454.0
legs1_segments0_seatsAvailable: 3737.0
legs0_segments0_seatsAvailable: 3721.0
legs0_arrivalAt_hour: 3577.0
legs1_arrivalAt_hour: 3280.0
legs0_departureAt_hour: 3152.0
legs1_departureAt_hour: 2996.0
corporateTariffCode: 2835.0
taxes: 2525.0
legs1_departureAt_weekday: 1990.0
legs0_departureAt_weekday: 1817.0
legs0_arrivalAt_weekday: 1588.0
legs1_arrivalAt_weekday: 1501.0
frequentFlyer: 1446.0
legs1_duration: 1376.0
legs0_duration: 1354.0


In [17]:
import pandas as pd

# 先準備特徵名稱
feature_names = list(X.columns)

# 取三種重要性
importance_types = ["weight", "gain", "cover"]
importance_all = {}

for imp_type in importance_types:
    imp_raw = xgb_model.get_score(importance_type=imp_type)
    imp_named = {}
    for k, v in imp_raw.items():
        idx = int(k[1:])
        real_name = feature_names[idx]
        imp_named[real_name] = v
    # 排序
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[imp_type] = sorted_imp

# 把三個榜單放成DataFrame方便比對
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover_rank"])
df_cover["cover_rank_pos"] = df_cover.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
    .merge(df_cover, on="feature", how="outer")
)

# 把不存在的rank補大數字
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

# 計算「三個榜單中最早出現的位置」
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# 取前50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank", "cover_rank"]])

# 如果想輸出CSV
top50.to_csv("feature_importance_top50.csv", index=False)
print("✅ 已輸出 feature_importance_top50.csv")


                                        feature  weight_rank  gain_rank  \
76                                   totalPrice       8578.0   4.235002   
22                   legs0_segments0_cabinClass        109.0  42.103062   
1                                     companyID       6165.0   3.913192   
34   legs0_segments1_departureFrom_airport_iata          1.0  41.605595   
26        legs0_segments0_marketingCarrier_code        153.0   3.633880   
62   legs1_segments1_departureFrom_airport_iata          2.0   7.080189   
25                 legs0_segments0_flightNumber       6092.0   4.140559   
0                              both_legs_direct         35.0  21.166515   
60  legs1_segments1_arrivalTo_airport_city_iata          1.0  18.557640   
55                 legs1_segments0_flightNumber       5554.0   4.186655   
16                           legs0_main_carrier         22.0   3.867804   
3                         days_before_departure       5454.0   3.778232   
20    legs0_segments0_bag

In [ ]:

data_xgb = X.with_columns([(pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32) for c in cat_features_final])

n1 = 16487352 # split train to train and val (10%) in time
n2 = train.height
data_xgb_tr, data_xgb_va, data_xgb_te = data_xgb[:n1], data_xgb[n1:n2], data_xgb[n2:]
y_tr, y_va, y_te = y[:n1], y[n1:n2], y[n2:]
groups_tr, groups_va, groups_te = groups[:n1], groups[n1:n2], groups[n2:]

group_sizes_tr = groups_tr.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
group_sizes_va = groups_va.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
group_sizes_te = groups_te.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
dtrain = xgb.DMatrix(data_xgb_tr, label=y_tr, group=group_sizes_tr, feature_names=data_xgb.columns)
dval   = xgb.DMatrix(data_xgb_va, label=y_va, group=group_sizes_va, feature_names=data_xgb.columns)
dtest  = xgb.DMatrix(data_xgb_te, label=y_te, group=group_sizes_te, feature_names=data_xgb.columns)

Id,ranker_id,selected,profileId,requestDate,miniRules0_percentage,miniRules1_percentage,frequentFlyer,pricingInfo_passengerCount
i64,str,i64,i64,datetime[ns],f64,f64,i32,i64
0,"""98ce0dabf6964640b63079fbafd42c…",1,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
1,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
2,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
3,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
4,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
…,…,…,…,…,…,…,…,…
18146427,"""88f8c53a28bf4f438941fd67338009…",0,3046852,2024-10-29 12:46:20,0.0,0.0,371,1
18146428,"""88f8c53a28bf4f438941fd67338009…",0,3046852,2024-10-29 12:46:20,0.0,0.0,371,1
18146429,"""88f8c53a28bf4f438941fd67338009…",0,3046852,2024-10-29 12:46:20,0.0,0.0,371,1


# Model

In [7]:
lgbm_params = {
    "boosting_type": "gbdt",
    "colsample_bytree": 0.5242042724303907,
    "learning_rate": 0.014470794293130388,
    "min_child_samples": 25,
    "min_child_weight": 0.1936457311991661,
    "n_estimators": 441,
    "n_jobs": -1,
    "num_leaves": 15,
    "max_depth": 12,
    "random_state": 42,
    "reg_alpha": 76.69015407123774,
    "reg_lambda": 78.57981723239948,
    "subsample": 0.35497610282716086,
    "verbose": -1
}

lgbm_goss_params = {
    "boosting_type": "goss",
    "colsample_bytree": 0.32266516869045214,
    "learning_rate": 0.013684657681610528,
    "min_child_samples": 16,
    "min_child_weight": 0.652800548618323,
    "n_estimators": 268,
    "n_jobs": -1,
    "num_leaves": 20,
    "max_depth": 6,
    "random_state": 42,
    "reg_alpha": 24.43093150663448,
    "reg_lambda": 39.81794248056326,
    "subsample": 0.21026644887863555,
    "verbose": -1
}

xgb_params = {
    "tree_method": "hist",
    "device": "gpu",
    "colsample_bylevel": 0.4778,
    "colsample_bynode": 0.3628,
    "colsample_bytree": 0.7107,
    "gamma": 1.7095,
    "learning_rate": 0.02213,
    "max_depth": 20,
    "max_leaves": 12,
    "min_child_weight": 16,
    "n_estimators": 1667,
    "subsample": 0.06567,
    "reg_alpha": 39.3524,
    "reg_lambda": 75.4484,
    "verbosity": 0,
    "random_state": 42,
    "n_jobs": -1
}

catboost_params = {
    "iterations": 1000,
    "depth": 6,  # 控制模型複雜度
    "learning_rate": 0.02,
    "l2_leaf_reg": 75.0,  # 對應 LGBM 的 reg_lambda 高值
    "random_strength": 42.0,  # 對應 XGB 的 gamma
    "bagging_temperature": 1.0,  # 強化樣本多樣性
    "subsample": 0.35,  # 對齊 LGBM 的 subsample
    "rsm": 0.52,  # 對齊 colsample_bytree
    "min_data_in_leaf": 25,  # 對應 min_child_samples
    "early_stopping_rounds": 50,
    "verbose": 0,
}



In [8]:
val_scores = {}
oof_preds = {}
test_preds = {}
train_scores = {}

In [9]:
xgb_trainer = Trainer(XGBRegressor(**xgb_params), models_folder= f"{CFG.sample_sub_path}/xgb")

oof_preds["XGBoost"], test_preds["XGBoost"], val_scores["XGBoost"], train_scores["XGBoost"] = xgb_trainer.fit_predict(X, y, X_test)

Training XGBRegressor


🧩 Fold 0
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//xgb/fold0_full_data/XGBRegressor.pkl
  ✅ full_data - val_pearson: 0.146689 - train_pearson: 0.658291
  ⏳ Training slice: last_75pct, samples: 350591
  📦 Saved last_75pct model to output/UMAP/k-fold/no_test//xgb/fold0_last_75pct/XGBRegressor.pkl
  ✅ last_75pct - val_pearson: 0.147851 - train_pearson: 0.657555
  ⏳ Training slice: last_50pct, samples: 262944
  📦 Saved last_50pct model to output/UMAP/k-fold/no_test//xgb/fold0_last_50pct/XGBRegressor.pkl
  ✅ last_50pct - val_pearson: 0.105701 - train_pearson: 0.684429

🧩 Fold 1
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//xgb/fold1_full_data/XGBRegressor.pkl
  ✅ full_data - val_pearson: 0.125696 - train_pearson: 0.642596
  ⏳ Training slice: last_75pct, samples: 219120
  📦 Saved last_75pct model to output/UMAP/k-fold/no_test//xgb/fold1_last_75pct/XG

In [39]:
xgb_trainer = Trainer(XGBRegressor(**xgb_params), models_folder= f"{CFG.sample_sub_path}/xgb")

oof_preds["XGBoost"], test_preds["XGBoost"], val_scores["XGBoost"], train_scores["XGBoost"] = xgb_trainer.fit_predict(X, y, X_test)

Training XGBRegressor


🧩 Fold 0
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//xgb/fold0_full_data/XGBRegressor.pkl
  ✅ full_data - val_pearson: 0.151434 - train_pearson: 0.622458
  ⏳ Training slice: last_75pct, samples: 350591
  📦 Saved last_75pct model to output/UMAP/k-fold/no_test//xgb/fold0_last_75pct/XGBRegressor.pkl
  ✅ last_75pct - val_pearson: 0.138200 - train_pearson: 0.621360
  ⏳ Training slice: last_50pct, samples: 262944
  📦 Saved last_50pct model to output/UMAP/k-fold/no_test//xgb/fold0_last_50pct/XGBRegressor.pkl
  ✅ last_50pct - val_pearson: 0.133944 - train_pearson: 0.654433

🧩 Fold 1
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//xgb/fold1_full_data/XGBRegressor.pkl
  ✅ full_data - val_pearson: 0.112764 - train_pearson: 0.601977
  ⏳ Training slice: last_75pct, samples: 219120
  📦 Saved last_75pct model to output/UMAP/k-fold/no_test//xgb/fold1_last_75pct/XG

In [30]:
sub_ens = pd.DataFrame({
    'ID': test_ids,
    'prediction': test_preds["XGBoost"]
})
ens_name = 'submission_try_new_selected_no_test_UMAP.csv'
sub_ens.to_csv(os.path.join(CFG.sample_sub_path, ens_name), index=False)
print(f"📦 Saved ensemble submission to {ens_name}")


📦 Saved ensemble submission to submission_try_new_selected_no_test_UMAP.csv


In [40]:
lgbm_trainer = Trainer(
    LGBMRegressor(**lgbm_params),
    models_folder= f"{CFG.sample_sub_path}/lgbm_gbdt"
)

oof_preds["LightGBM (gbdt)"], test_preds["LightGBM (gbdt)"], val_scores["LightGBM (gbdt)"], train_scores["LightGBM (gbdt)"] = lgbm_trainer.fit_predict(X, y, X_test)

Training LGBMRegressor


🧩 Fold 0
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold0_full_data/LGBMRegressor.pkl
  ✅ full_data - val_pearson: 0.088601 - train_pearson: 0.624664
  ⏳ Training slice: last_75pct, samples: 350591
  📦 Saved last_75pct model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold0_last_75pct/LGBMRegressor.pkl
  ✅ last_75pct - val_pearson: 0.096252 - train_pearson: 0.625570
  ⏳ Training slice: last_50pct, samples: 262944
  📦 Saved last_50pct model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold0_last_50pct/LGBMRegressor.pkl
  ✅ last_50pct - val_pearson: 0.092069 - train_pearson: 0.654027

🧩 Fold 1
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold1_full_data/LGBMRegressor.pkl
  ✅ full_data - val_pearson: 0.084856 - train_pearson: 0.618298
  ⏳ Training slice: last_75pct, samples: 219120
  📦 Saved last_75pct model to output/UMAP/k-fold/no_

In [23]:
lgbm_trainer = Trainer(
    LGBMRegressor(**lgbm_params),
    models_folder= f"{CFG.sample_sub_path}/lgbm_gbdt"
)

oof_preds["LightGBM (gbdt)"], test_preds["LightGBM (gbdt)"], val_scores["LightGBM (gbdt)"], train_scores["LightGBM (gbdt)"] = lgbm_trainer.fit_predict(X, y, X_test)

Training LGBMRegressor


🧩 Fold 0
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold0_full_data/LGBMRegressor.pkl
  ✅ full_data - val_pearson: 0.099182 - train_pearson: 0.625195
  ⏳ Training slice: last_75pct, samples: 350591
  📦 Saved last_75pct model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold0_last_75pct/LGBMRegressor.pkl
  ✅ last_75pct - val_pearson: 0.099182 - train_pearson: 0.625195
  ⏳ Training slice: last_50pct, samples: 262944
  📦 Saved last_50pct model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold0_last_50pct/LGBMRegressor.pkl
  ✅ last_50pct - val_pearson: 0.092069 - train_pearson: 0.654027

🧩 Fold 1
  ⏳ Training slice: full_data, samples: 350591
  📦 Saved full_data model to output/UMAP/k-fold/no_test//lgbm_gbdt/fold1_full_data/LGBMRegressor.pkl
  ✅ full_data - val_pearson: 0.080179 - train_pearson: 0.621754
  ⏳ Training slice: last_75pct, samples: 219120
  📦 Saved last_75pct model to output/UMAP/k-fold/no_

In [24]:
sub_ens = pd.DataFrame({
    'ID': test_ids,
    'prediction': test_preds["LightGBM (gbdt)"]
})
ens_name = 'submission_try_new_selected_no_test_UMAP.csv'
sub_ens.to_csv(os.path.join(CFG.sample_sub_path, ens_name), index=False)
print(f"📦 Saved ensemble submission to {ens_name}")


📦 Saved ensemble submission to submission_try_new_selected_no_test_UMAP.csv


In [ ]:
lgbm_goss_trainer = Trainer(LGBMRegressor(**lgbm_goss_params),     models_folder= f"{CFG.sample_sub_path}/goss_lgbm")

oof_preds["LightGBM (goss)"], test_preds["LightGBM (goss)"], val_scores["LightGBM (goss)"], train_scores["LightGBM (goss)"] = lgbm_goss_trainer.fit_predict(X, y, X_test)

Training LGBMRegressor

📦 Saved fold 0 model to output/k-fold/other_selected_25/ensemble//goss_lgbm/fold0/LGBMRegressor.pkl
--- Fold 0 - val_pearson: 0.180462 - train_pearson: 0.207905
📦 Saved fold 1 model to output/k-fold/other_selected_25/ensemble//goss_lgbm/fold1/LGBMRegressor.pkl
--- Fold 1 - val_pearson: 0.113101 - train_pearson: 0.222472
📦 Saved fold 2 model to output/k-fold/other_selected_25/ensemble//goss_lgbm/fold2/LGBMRegressor.pkl
--- Fold 2 - val_pearson: 0.068248 - train_pearson: 0.223433
📦 Saved fold 3 model to output/k-fold/other_selected_25/ensemble//goss_lgbm/fold3/LGBMRegressor.pkl
--- Fold 3 - val_pearson: 0.148354 - train_pearson: 0.210567
📦 Saved fold 4 model to output/k-fold/other_selected_25/ensemble//goss_lgbm/fold4/LGBMRegressor.pkl
--- Fold 4 - val_pearson: 0.091773 - train_pearson: 0.226333

------ Overall Score: 0.100684 - Mean val_pearson: 0.120388 ± 0.039954 - Mean train_pearson: 0.218142 ± 0.007430 ------



In [21]:
xgb_trainer = Trainer(XGBRegressor(**xgb_params), models_folder= f"{CFG.sample_sub_path}/xgb")

oof_preds["XGBoost"], test_preds["XGBoost"], val_scores["XGBoost"], train_scores["XGBoost"] = xgb_trainer.fit_predict(X, y, test)

Training XGBRegressor



XGBoostError: [22:08:45] /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/src/data/gradient_index.h:100: Check failed: valid: Input data contains `inf` or a value too large, while `missing` is not set to `inf`
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000015d67a0f0 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000015d81b618 void xgboost::GHistIndexMatrix::SetIndexData<xgboost::data::ColumnarAdapterBatch, unsigned int, void xgboost::GHistIndexMatrix::PushBatchImpl<xgboost::data::ColumnarAdapterBatch, xgboost::data::IsValidFunctor&>(int, xgboost::data::ColumnarAdapterBatch const&, unsigned long, xgboost::data::IsValidFunctor&, xgboost::common::Span<xgboost::FeatureType const, 18446744073709551615ul>)::'lambda'(xgboost::data::ColumnarAdapterBatch, xgboost::data::IsValidFunctor&), xgboost::data::IsValidFunctor&>(xgboost::common::Span<xgboost::data::IsValidFunctor&, 18446744073709551615ul

In [ ]:
sub_ens = pd.DataFrame({
    'ID': test_ids,
    'prediction': test_preds["Ridge"]
})
ens_name = 'submission_try_umap.csv'
sub_ens.to_csv(os.path.join(CFG.sample_sub_path, ens_name), index=False)
print(f"📦 Saved ensemble submission to {ens_name}")


In [22]:
ridge_model = Ridge(alpha=1.0, random_state=CFG.seed)
ridge_trainer = Trainer(ridge_model, models_folder=f"{CFG.sample_sub_path}/ridge")
oof_preds["Ridge"], test_preds["Ridge"], val_scores["Ridge"], train_scores["Ridge"] = ridge_trainer.fit_predict(X, y, test)


Training Ridge



ValueError: Input X contains NaN.
Ridge does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.linear_model import Ridge, BayesianRidge
from catboost import CatBoostRegressor

ridge_model = Ridge(alpha=1.0, random_state=CFG.seed)
catboost_model = CatBoostRegressor(verbose=0, random_state=CFG.seed)
bayes_model = BayesianRidge()

ridge_trainer = Trainer(ridge_model, models_folder=f"{CFG.sample_sub_path}/ridge")
catboost_trainer = Trainer(catboost_model, models_folder=f"{CFG.sample_sub_path}/catboost")
bayes_trainer = Trainer(bayes_model, models_folder=f"{CFG.sample_sub_path}/bayesridge")

# 執行訓練與預測
oof_preds["Ridge"], test_preds["Ridge"], val_scores["Ridge"], train_scores["Ridge"] = ridge_trainer.fit_predict(X, y, X_test)
oof_preds["CatBoost"], test_preds["CatBoost"], val_scores["CatBoost"], train_scores["CatBoost"] = catboost_trainer.fit_predict(X, y, X_test)
oof_preds["BayesRidge"], test_preds["BayesRidge"], scores["BayesRidge"] = bayes_trainer.fit_predict(X, y, X_test)


Training Ridge

📦 Saved fold 0 model to output/k-fold/other_selected_25/ensemble//ridge/fold0/Ridge.pkl
--- Fold 0 - val_pearson: 0.158685 - train_pearson: 0.141299
📦 Saved fold 1 model to output/k-fold/other_selected_25/ensemble//ridge/fold1/Ridge.pkl
--- Fold 1 - val_pearson: 0.083304 - train_pearson: 0.159373
📦 Saved fold 2 model to output/k-fold/other_selected_25/ensemble//ridge/fold2/Ridge.pkl
--- Fold 2 - val_pearson: 0.057616 - train_pearson: 0.168464
📦 Saved fold 3 model to output/k-fold/other_selected_25/ensemble//ridge/fold3/Ridge.pkl
--- Fold 3 - val_pearson: 0.122938 - train_pearson: 0.148199
📦 Saved fold 4 model to output/k-fold/other_selected_25/ensemble//ridge/fold4/Ridge.pkl
--- Fold 4 - val_pearson: 0.100707 - train_pearson: 0.168487

------ Overall Score: 0.093739 - Mean val_pearson: 0.104650 ± 0.034461 - Mean train_pearson: 0.157165 ± 0.010889 ------

Training CatBoostRegressor

📦 Saved fold 0 model to output/k-fold/other_selected_25/ensemble//catboost/fold0/CatBoost

In [ ]:
# normalize predictions to zero-mean before ensembling
def normalize_preds(preds_dict):
    return {k: v - np.mean(v) for k, v in preds_dict.items()}
nor_oof_preds = normalize_preds(oof_preds)
nor_test_preds = normalize_preds(test_preds)


In [ ]:
import os
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
from scripts.config import CFG
import json


def save_config_as_json(params_dict, selected_feature_sets, seeds, save_dir):
    config = {
        "model_params": params_dict,
        "selected_feature_sets": selected_feature_sets,
        "seeds": seeds
    }
    os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, "config.json"), "w") as f:
        json.dump(config, f, indent=4)

def run_all_models_with_seeds(train_df, test_df, target_col, selected_feature_sets: dict, seeds: list,
                              lgbm_params, lgbm_goss_params, xgb_params, catboost_params):

    # 保存參數與設定
    save_config_as_json({
        "lgbm_params": lgbm_params,
        "lgbm_goss_params": lgbm_goss_params,
        "xgb_params": xgb_params,
        "catboost_params": catboost_params
    }, selected_feature_sets, seeds, CFG.sample_sub_path)

    ens_train_frames = []
    ens_test_frames = []

    for feat_name, selected_features in selected_feature_sets.items():
        for seed in seeds:
            folder = os.path.join(CFG.sample_sub_path, f"{feat_name}/{seed}")
            os.makedirs(folder, exist_ok=True)

            train = train_df[selected_features + [target_col]].copy()
            test = test_df[selected_features].copy()
            X = train.drop(target_col, axis=1)
            y = train[target_col]
            X_test = test

            oof_preds, test_preds = {}, {}

            # 動態更新 seed 到參數
            lgbm_params_seeded = lgbm_params.copy()
            lgbm_params_seeded["random_state"] = seed

            lgbm_goss_params_seeded = lgbm_goss_params.copy()
            lgbm_goss_params_seeded["random_state"] = seed

            xgb_params_seeded = xgb_params.copy()
            xgb_params_seeded["random_state"] = seed
            
            catboost_params_seeded = catboost_params.copy()
            catboost_params_seeded["random_state"] = seed
            
            models = {
                "lgbm_gbdt": LGBMRegressor(**lgbm_params_seeded),
                "lgbm_goss": LGBMRegressor(**lgbm_goss_params_seeded),
                "xgb": XGBRegressor(**xgb_params_seeded),
                "ridge": Ridge(random_state=seed),
                "catboost": CatBoostRegressor(**catboost_params_seeded),
            }


            for name, model in models.items():
                model_folder = os.path.join(folder, name)
                trainer = Trainer(model, models_folder=model_folder)
                oof, test_pred, _, _ = trainer.fit_predict(X, y, X_test)
                oof_preds[name] = oof
                test_preds[name] = test_pred

            # Normalize predictions
            def normalize(preds):
                return {k: v - np.mean(v) for k, v in preds.items()}

            norm_oof = normalize(oof_preds)
            norm_test = normalize(test_preds)

            oof_df = pd.DataFrame({
                f"{k}_{feat_name}_seed{seed}": v for k, v in norm_oof.items()
            })

            test_df_part = pd.DataFrame({
                f"{k}_{feat_name}_seed{seed}": v for k, v in norm_test.items()
            })

            ens_train_frames.append(oof_df)
            ens_test_frames.append(test_df_part)

    X_ens = pd.concat(ens_train_frames, axis=1)
    X_test_ens = pd.concat(ens_test_frames, axis=1)
    return X_ens, X_test_ens


CFG.sample_sub_path = "output/k-fold/all_different_data_sand_seed/ensemble/"

other_selected_25 = list(dict.fromkeys([
    "X863", "X856", "X344", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
    "X415", "X345", "X137", "X855", "X174", "X302", "X178", "X532", "X168", "X612",
    "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"
]))
best_21 = list(dict.fromkeys([
    'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume',
    'X695', 'X21', 'X603', 'X855', 'X285', 'X344', 'X345',
    'X444', 'X532', 'X856', 'X862', 'X863', 'X873', 'X887','X40', 'X861',]))
best_21_and_others = list(dict.fromkeys([
    'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume',
    'X695', 'X21', 'X603', 'X855', 'X285', 'X344', 'X345',
    'X444', 'X532', 'X856', 'X862', 'X863', 'X873', 'X887','X40', 'X861',
    'X598', 'X385', 'X168', 'X174', 'X178', 'X302', 'X415', 'X612', 'X852', 'X860', 'X674', 'X137']))


X, X_test = run_all_models_with_seeds(train, test, CFG.target, {
    "other_selected_25": other_selected_25,
    "selected_21"   : best_21,
    "other&my_selected_33": best_21_and_others
}, seeds=[42, 1337, 1024], lgbm_params=lgbm_params, lgbm_goss_params=lgbm_goss_params, xgb_params=xgb_params, catboost_params=catboost_params)


In [9]:
# Step 1: 為每組特徵調參

import optuna
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import numpy as np
import warnings

warnings.filterwarnings("ignore")


def tune_all_models(train_df, target_col, selected_features, n_trials=30):
    X = train_df[selected_features]
    y = train_df[target_col]

    def evaluate_model(model, X, y, n_splits=5, alpha=1):
        kf = KFold(n_splits=n_splits, shuffle=False)
        val_scores, train_scores = [], []
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            model.fit(X_train, y_train)
            val_pred = model.predict(X_val)
            train_pred = model.predict(X_train)
            val_scores.append(pearsonr(val_pred, y_val)[0])
            train_scores.append(pearsonr(train_pred, y_train)[0])
        val_mean = np.mean(val_scores)
        train_mean = np.mean(train_scores)
        score = val_mean - alpha * (train_mean - val_mean)
        return score


    def objective_lgbm(trial):
        params = {
            "boosting_type": "gbdt",
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.05, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 15, 64),
            "max_depth": trial.suggest_int("max_depth", 4, 16),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 1.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.3, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 100.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 100.0, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
            "random_state": 42,
            "n_jobs": -1,
            "verbose": -1
        }
        model = LGBMRegressor(**params)
        return evaluate_model(model, X, y)

    def objective_lgbm_goss(trial):
        params = {
            "boosting_type": "goss",
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.05, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 10, 30),
            "max_depth": trial.suggest_int("max_depth", 4, 12),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 1.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.2, 0.8),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 50.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 50.0, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
            "random_state": 42,
            "n_jobs": -1,
            "verbose": -1
        }
        model = LGBMRegressor(**params)
        return evaluate_model(model, X, y)

    def objective_xgb(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.05, log=True),
            "max_depth": trial.suggest_int("max_depth", 4, 16),
            "max_leaves": trial.suggest_int("max_leaves", 4, 32),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 30),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "subsample": trial.suggest_float("subsample", 0.3, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 1.0),
            "colsample_bynode": trial.suggest_float("colsample_bynode", 0.3, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 100.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 100.0, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
            "n_jobs": -1,
            "random_state": 42,
            "verbosity": 0,
        }
        model = XGBRegressor(**params)
        return evaluate_model(model, X, y)

    def objective_catboost(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 100, 2000),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 100.0, log=True),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
            "subsample": trial.suggest_float("subsample", 0.3, 1.0),
            "rsm": trial.suggest_float("rsm", 0.3, 1.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 30),
            "random_strength": trial.suggest_float("random_strength", 1.0, 50.0),
            "early_stopping_rounds": 30,
            "verbose": 0,
            "random_state": 42,
        }
        model = CatBoostRegressor(**params)
        return evaluate_model(model, X, y)

    print("🔍 Tuning LGBM (gbdt)...")
    lgbm_study = optuna.create_study(direction="maximize")
    lgbm_study.optimize(objective_lgbm, n_trials=n_trials)

    print("🔍 Tuning LGBM (goss)...")
    goss_study = optuna.create_study(direction="maximize")
    goss_study.optimize(objective_lgbm_goss, n_trials=n_trials)

    print("🔍 Tuning XGBoost...")
    xgb_study = optuna.create_study(direction="maximize")
    xgb_study.optimize(objective_xgb, n_trials=n_trials)

    print("🔍 Tuning CatBoost...")
    cat_study = optuna.create_study(direction="maximize")
    cat_study.optimize(objective_catboost, n_trials=n_trials)

    return {
        "lgbm_params": lgbm_study.best_params,
        "lgbm_goss_params": goss_study.best_params,
        "xgb_params": xgb_study.best_params,
        "catboost_params": cat_study.best_params
    }



all_best_params = {
    name: tune_all_models(train_df=train, target_col=CFG.target, selected_features=feat_list)
    for name, feat_list in {
    "selected_21"   : best_21,
    "other&my_selected_33": best_21_and_others
    }.items()
}

[I 2025-05-31 17:37:21,303] A new study created in memory with name: no-name-9c25bbaa-60d9-4d8d-92b0-08655006e825


🔍 Tuning LGBM (gbdt)...


[I 2025-05-31 17:37:56,193] Trial 0 finished with value: -0.2665519214689309 and parameters: {'learning_rate': 0.001657932720775217, 'num_leaves': 35, 'max_depth': 13, 'min_child_samples': 18, 'min_child_weight': 0.00835840364628969, 'subsample': 0.34277328240621585, 'colsample_bytree': 0.46423490143339535, 'reg_alpha': 6.974970696338066, 'reg_lambda': 34.25138210663246, 'n_estimators': 288}. Best is trial 0 with value: -0.2665519214689309.
[I 2025-05-31 17:39:40,966] Trial 1 finished with value: -0.3615868409906824 and parameters: {'learning_rate': 0.003966135442950583, 'num_leaves': 44, 'max_depth': 8, 'min_child_samples': 25, 'min_child_weight': 0.0015157000752770153, 'subsample': 0.4299968122912713, 'colsample_bytree': 0.9793743214547576, 'reg_alpha': 11.334528844804153, 'reg_lambda': 38.00436867219635, 'n_estimators': 887}. Best is trial 0 with value: -0.2665519214689309.
[I 2025-05-31 17:44:59,816] Trial 2 finished with value: -0.5988195457125005 and parameters: {'learning_rate':

: 

In [ ]:
all_best_params["other_selected_25"] = {
    "lgbm_params": {
        "learning_rate": 0.001637813850082008,
        "num_leaves": 10,
        "max_depth": 11,
        "min_child_samples": 5,
        "min_child_weight": 0.016271558733787436,
        "subsample": 0.6593613083987416,
        "colsample_bytree": 0.4575721609314488,
        "reg_alpha": 1.699018742860479,
        "reg_lambda": 0.25754595543778913,
        "n_estimators": 323,
        "random_state": 42,
        "n_jobs": -1,
        "boosting_type": "gbdt",
        "verbose": -1,
    },
    "lgbm_goss_params": {
        "learning_rate": 0.0010158433360328654,
        "num_leaves": 15,
        "max_depth": 4,
        "min_child_samples": 22,
        "min_child_weight": 0.0011960681689680813,
        "subsample": 0.9788004638386685,
        "colsample_bytree": 0.4336760144773911,
        "reg_alpha": 0.4229242901001258,
        "reg_lambda": 94.77537152236522,
        "n_estimators": 112,
        "random_state": 42,
        "n_jobs": -1,
        "boosting_type": "goss",
        "verbose": -1,
    },
    "xgb_params": {
        "learning_rate": 0.001889595449145299,
        "max_depth": 5,
        "max_leaves": 4,
        "min_child_weight": 19,
        "gamma": 0.4730991659940904,
        "subsample": 0.9997261759454115,
        "colsample_bytree": 0.3180029675660578,
        "colsample_bylevel": 0.8413518907705161,
        "colsample_bynode": 0.33111438641088414,
        "reg_alpha": 39.861565568274074,
        "reg_lambda": 0.10988448724798101,
        "n_estimators": 711,
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": 0
    },
    "catboost_params": {
        "iterations": 144,
        "depth": 4,
        "learning_rate": 0.011809472839755976,
        "l2_leaf_reg": 28.723858155851214,
        "bagging_temperature": 0.5266938712453599,
        "subsample": 0.46783583928704287,
        "rsm": 0.8149725374297446,
        "min_data_in_leaf": 25,
        "random_strength": 23.183405644995034,
        "early_stopping_rounds": 30,
        "verbose": 0,
        "random_state": 42,
    }
}


In [ ]:
def run_all_models_with_seeds_from_best_params(
    train_df,
    test_df,
    target_col,
    selected_feature_sets: dict,
    seeds: list,
    all_best_params: dict  # <-- 傳入每個 feature_set 的最佳參數
):
    ens_train_frames = []
    ens_test_frames = []

    for feat_name, selected_features in selected_feature_sets.items():
        best_params = all_best_params[feat_name]

        # 保存該 feature_set 的設定
        save_config_as_json(
            best_params,
            {feat_name: selected_features},
            seeds,
            os.path.join(CFG.sample_sub_path, feat_name)
        )

        for seed in seeds:
            folder = os.path.join(CFG.sample_sub_path, f"{feat_name}/{seed}")
            os.makedirs(folder, exist_ok=True)

            train = train_df[selected_features + [target_col]].copy()
            test = test_df[selected_features].copy()
            X = train.drop(target_col, axis=1)
            y = train[target_col]
            X_test = test

            oof_preds, test_preds = {}, {}

            # 注入對應 seed
            lgbm_params_seeded = best_params["lgbm_params"].copy()
            lgbm_params_seeded["random_state"] = seed

            lgbm_goss_params_seeded = best_params["lgbm_goss_params"].copy()
            lgbm_goss_params_seeded["random_state"] = seed

            xgb_params_seeded = best_params["xgb_params"].copy()
            xgb_params_seeded["random_state"] = seed

            catboost_params_seeded = best_params["catboost_params"].copy()
            catboost_params_seeded["random_state"] = seed

            models = {
                "lgbm_gbdt": LGBMRegressor(**lgbm_params_seeded),
                "lgbm_goss": LGBMRegressor(**lgbm_goss_params_seeded),
                "xgb": XGBRegressor(**xgb_params_seeded),
                "ridge": Ridge(random_state=seed),
                "catboost": CatBoostRegressor(**catboost_params_seeded),
            }

            for name, model in models.items():
                model_folder = os.path.join(folder, name)
                trainer = Trainer(model, models_folder=model_folder)
                oof, test_pred, _, _ = trainer.fit_predict(X, y, X_test)
                oof_preds[name] = oof
                test_preds[name] = test_pred

            # Normalize predictions
            def normalize(preds):
                return {k: v - np.mean(v) for k, v in preds.items()}

            norm_oof = normalize(oof_preds)
            norm_test = normalize(test_preds)

            oof_df = pd.DataFrame({
                f"{k}_{feat_name}_seed{seed}": v for k, v in norm_oof.items()
            })

            test_df_part = pd.DataFrame({
                f"{k}_{feat_name}_seed{seed}": v for k, v in norm_test.items()
            })

            ens_train_frames.append(oof_df)
            ens_test_frames.append(test_df_part)

    X_ens = pd.concat(ens_train_frames, axis=1)
    X_test_ens = pd.concat(ens_test_frames, axis=1)
    return X_ens, X_test_ens



CFG.sample_sub_path = "output/k-fold/all_different_data_sand_seed_v2/ensemble/"
X, X_test = run_all_models_with_seeds_from_best_params(
    train, test, CFG.target,
    {
        "other_selected_25": other_selected_25,
        "selected_21": best_21,
        "other&my_selected_33": best_21_and_others
    },
    seeds=[42, 1337, 1024],
    all_best_params=all_best_params
)


In [ ]:
X = pd.DataFrame(nor_oof_preds)
X_test = pd.DataFrame(nor_test_preds)

In [ ]:
# 儲存資料
X.to_parquet(os.path.join(CFG.sample_sub_path, "stacked_train_X.parquet"), index=False)
X_test.to_parquet(os.path.join(CFG.sample_sub_path, "stacked_test_X.parquet"), index=False)

In [ ]:
import optuna
from sklearn.linear_model import Ridge
from sklearn.base import clone
from scipy.stats import pearsonr
import gc

def objective(trial):
    params = {
        "random_state": CFG.seed,
        "alpha": trial.suggest_float("alpha", 0, 1000),
        "tol": trial.suggest_float("tol", 1e-6, 1e-2),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        "positive": trial.suggest_categorical("positive", [True, False])
    }

    model = Ridge(**params)
    scores = []

    split = KFold(n_splits=CFG.n_folds, shuffle=False).split(X, y)
    for fold_idx, (train_idx, val_idx) in enumerate(split):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        clf = clone(model)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        score = pearsonr(y_val, y_pred)[0]
        scores.append(score)

    avg_score = np.mean(scores)
    return avg_score

if CFG.run_optuna:
    sampler = optuna.samplers.TPESampler(seed=CFG.seed, multivariate=True)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=CFG.n_optuna_trials, n_jobs=-1, catch=(ValueError,))
    best_params = study.best_params

    ridge_params = {
        "random_state": CFG.seed,
        "alpha": best_params["alpha"],
        "tol": best_params["tol"],
        "fit_intercept": best_params["fit_intercept"],
        "positive": best_params["positive"]
    }
else:
    ridge_params = {
        "random_state": CFG.seed
    }


[I 2025-05-31 04:25:19,408] A new study created in memory with name: no-name-88d3063c-ba61-42db-b318-3690a0429721
[I 2025-05-31 04:25:26,535] Trial 1 finished with value: 0.13064979477501532 and parameters: {'alpha': 68.30976363852781, 'tol': 0.0007066106203514526, 'fit_intercept': False, 'positive': False}. Best is trial 1 with value: 0.13064979477501532.
[I 2025-05-31 04:25:26,547] Trial 5 finished with value: 0.1427723543510045 and parameters: {'alpha': 672.8733603617318, 'tol': 4.148494110618029e-05, 'fit_intercept': True, 'positive': False}. Best is trial 5 with value: 0.1427723543510045.
[I 2025-05-31 04:25:26,560] Trial 3 finished with value: 0.14270257063480707 and parameters: {'alpha': 667.0760734062089, 'tol': 0.007594589914168868, 'fit_intercept': True, 'positive': False}. Best is trial 5 with value: 0.1427723543510045.
[I 2025-05-31 04:25:26,572] Trial 0 finished with value: 0.13860206879727455 and parameters: {'alpha': 385.5633850632768, 'tol': 0.006964927334096522, 'fit_i

In [39]:
ridge_params

{'random_state': 42,
 'alpha': 408.8163672645777,
 'tol': 0.00011771640004788044,
 'fit_intercept': True,
 'positive': True}

In [14]:
ridge_params

{'random_state': 42,
 'alpha': 2.349295484010465,
 'tol': 0.0005740485460709731,
 'fit_intercept': False,
 'positive': True}

In [ ]:
ridge_trainer = Trainer(Ridge(**ridge_params), models_folder=f"{CFG.sample_sub_path}/metamodel_ridge_optuna/")
# X = pd.DataFrame(oof_preds)
# X_test = pd.DataFrame(test_preds)
_, ridge_test_preds, val_scores["Ridge (ensemble)"], train_scores["Ridge (ensemble)"] = ridge_trainer.fit_predict(X, y, X_test)

Training Ridge

📦 Saved fold 0 model to output/k-fold/all_different_data_sand_seed/ensemble//metamodel_ridge_optuna/fold0/Ridge.pkl
--- Fold 0 - val_pearson: 0.227517 - train_pearson: 0.145023
📦 Saved fold 1 model to output/k-fold/all_different_data_sand_seed/ensemble//metamodel_ridge_optuna/fold1/Ridge.pkl
--- Fold 1 - val_pearson: 0.154319 - train_pearson: 0.157349
📦 Saved fold 2 model to output/k-fold/all_different_data_sand_seed/ensemble//metamodel_ridge_optuna/fold2/Ridge.pkl
--- Fold 2 - val_pearson: 0.111058 - train_pearson: 0.164239
📦 Saved fold 3 model to output/k-fold/all_different_data_sand_seed/ensemble//metamodel_ridge_optuna/fold3/Ridge.pkl
--- Fold 3 - val_pearson: 0.220108 - train_pearson: 0.144857
📦 Saved fold 4 model to output/k-fold/all_different_data_sand_seed/ensemble//metamodel_ridge_optuna/fold4/Ridge.pkl
--- Fold 4 - val_pearson: 0.141035 - train_pearson: 0.179207

------ Overall Score: 0.134633 - Mean val_pearson: 0.170807 ± 0.045552 - Mean train_pearson: 0.158

In [ ]:
sub_ens = pd.DataFrame({
    'ID': test_ids,
    'prediction': ridge_test_preds
})
ens_name = 'submission_ensemble_no_MLP_ridge_op.csv'
sub_ens.to_csv(os.path.join(CFG.sample_sub_path, ens_name), index=False)
print(f"📦 Saved ensemble submission to {ens_name}")


📦 Saved ensemble submission to submission_ensemble_no_MLP_ridge_op.csv


In [ ]:
import optuna
from sklearn.neural_network import MLPRegressor

def objective(trial):
    hidden_units = trial.suggest_categorical("hidden_layer_sizes", [(6,), (8,), (8, 4), (10, 5), (12, 6, 3)])
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    solver = trial.suggest_categorical("solver", ["adam", "lbfgs"])
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size", 64, 2048, log=True)

    params = {
        "hidden_layer_sizes": hidden_units,
        "activation": activation,
        "solver": solver,
        "alpha": alpha,
        "learning_rate_init": learning_rate_init,
        "batch_size": batch_size,
        "max_iter": 200,
        "early_stopping": True,
        "random_state": CFG.seed
    }

    trainer = Trainer(MLPRegressor(**params), models_folder=f"{CFG.sample_sub_path}/metamodel_6_mlp_optuna")
    return trainer.tune(X, y)


if CFG.run_optuna:
    sampler = optuna.samplers.TPESampler(seed=CFG.seed, multivariate=True)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=CFG.n_optuna_trials, n_jobs=-1, catch=(ValueError,))
    best_params = study.best_params

    mlp_params = {
        "hidden_layer_sizes": best_params["hidden_layer_sizes"],
        "activation": best_params["activation"],
        "solver": best_params["solver"],
        "alpha": best_params["alpha"],
        "learning_rate_init": best_params["learning_rate_init"],
        "batch_size": 64,
        "max_iter": 200,
        "early_stopping": True,
        "random_state": CFG.seed
    }
else:
    mlp_params = {
        "hidden_layer_sizes": (8, 4),
        "activation": "relu",
        "solver": "adam",
        "alpha": 1e-3,
        "learning_rate_init": 1e-3,
        "batch_size": 64,
        "max_iter": 200,
        "early_stopping": True,
        "random_state": CFG.seed
    }

# 產生 MLP Trainer 並執行
mlp_trainer = Trainer(MLPRegressor(**mlp_params), models_folder=f"{CFG.sample_sub_path}/metamodel_6_mlp_optuna")
oof_preds["MLP"], test_preds["MLP"], scores["MLP"] = mlp_trainer.fit_predict(X, y, X_test)


[I 2025-05-30 21:01:20,064] A new study created in memory with name: no-name-73a0436e-badc-4639-ac2c-40640694e3f9


[I 2025-05-30 21:02:36,086] Trial 3 finished with value: 0.134202350233761 and parameters: {'hidden_layer_sizes': (6,), 'activation': 'tanh', 'solver': 'lbfgs', 'alpha': 0.003024536856435966, 'learning_rate_init': 0.00031541353228579515, 'batch_size': 346}. Best is trial 3 with value: 0.134202350233761.
[I 2025-05-30 21:02:57,921] Trial 1 finished with value: 0.1337708131462209 and parameters: {'hidden_layer_sizes': (8, 4), 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 0.005013483541110714, 'learning_rate_init': 0.002345802415777592, 'batch_size': 1130}. Best is trial 3 with value: 0.134202350233761.
[I 2025-05-30 21:03:39,626] Trial 0 finished with value: 0.13574429482257883 and parameters: {'hidden_layer_sizes': (10, 5), 'activation': 'tanh', 'solver': 'lbfgs', 'alpha': 3.7658628818285295e-05, 'learning_rate_init': 0.0001477580844823203, 'batch_size': 623}. Best is trial 0 with value: 0.13574429482257883.
[I 2025-05-30 21:03:43,476] Trial 8 finished with value: 0.1374459771240900

In [ ]:
mlp_params

{'hidden_layer_sizes': (6,),
 'activation': 'tanh',
 'solver': 'adam',
 'alpha': 0.07769191538624844,
 'learning_rate_init': 0.003347750729180181,
 'batch_size': 64,
 'max_iter': 200,
 'early_stopping': True,
 'random_state': 42}

In [ ]:
sub_ens = pd.DataFrame({
    'ID': test_ids,
    'prediction': test_preds["MLP"]
})
ens_name = 'submission_ensemble_6models_mlp_optuna.csv'
sub_ens.to_csv(os.path.join(CFG.sample_sub_path, ens_name), index=False)
print(f"📦 Saved ensemble submission to {ens_name}")


Training MLPRegressor

📦 Saved fold 0 model to output/k-fold/ensemble//metamodel_mlp/fold0/MLPRegressor.pkl
--- Fold 0 - Score: 0.070023
📦 Saved fold 1 model to output/k-fold/ensemble//metamodel_mlp/fold1/MLPRegressor.pkl
--- Fold 1 - Score: 0.109087
📦 Saved fold 2 model to output/k-fold/ensemble//metamodel_mlp/fold2/MLPRegressor.pkl
--- Fold 2 - Score: 0.037309
📦 Saved fold 3 model to output/k-fold/ensemble//metamodel_mlp/fold3/MLPRegressor.pkl
--- Fold 3 - Score: 0.133597
📦 Saved fold 4 model to output/k-fold/ensemble//metamodel_mlp/fold4/MLPRegressor.pkl
--- Fold 4 - Score: 0.109197

------ Overall Score: 0.077463 - Mean Score: 0.091843 ± 0.034035


In [19]:
# 儲存資料
X.to_parquet(os.path.join(CFG.sample_sub_path, "stacked_train_X.parquet"), index=False)
X_test.to_parquet(os.path.join(CFG.sample_sub_path, "stacked_test_X.parquet"), index=False)

In [ ]:
sub_ens = pd.DataFrame({
    'ID': test_ids,
    'prediction': ridge_test_preds
})
ens_name = 'submission_ensemble_6models_mlp_op.csv'
sub_ens.to_csv(os.path.join(CFG.sample_sub_path, ens_name), index=False)
print(f"📦 Saved ensemble submission to {ens_name}")


📦 Saved ensemble submission to submission_ensemble_6models_mlp_op.csv
